<a href="https://colab.research.google.com/github/charita-dotcom/colab/blob/main/earbio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content/gdrive/My Drive/Kaggle"

In [ ]:
%cd /content/gdrive/My Drive/Kaggle

/content/gdrive/My Drive/Kaggle


In [ ]:
pwd

'/content/gdrive/My Drive/Kaggle'

In [ ]:
!kaggle datasets download -d omarhatif/datasets-for-ear-detection-and-recognition

 98% 270M/275M [00:05<00:00, 64.7MB/s]
100% 275M/275M [00:05<00:00, 52.0MB/s]


In [ ]:
!ls

datasets-for-ear-detection-and-recognition.zip	kaggle.json


In [ ]:
!unzip \*.zip  && rm *.zip

Archive:  datasets-for-ear-detection-and-recognition.zip
  inflating: ear recognition datasets/Dataset1/info.txt  
  inflating: ear recognition datasets/Dataset1/test/000/61gr.png  
  inflating: ear recognition datasets/Dataset1/test/000/61rfb.png  
  inflating: ear recognition datasets/Dataset1/test/000/62gr.png  
  inflating: ear recognition datasets/Dataset1/test/000/62rfb.png  
  inflating: ear recognition datasets/Dataset1/test/000/63gr.png  
  inflating: ear recognition datasets/Dataset1/test/000/63rfb.png  
  inflating: ear recognition datasets/Dataset1/test/000/64gr.png  
  inflating: ear recognition datasets/Dataset1/test/000/64rfb.png  
  inflating: ear recognition datasets/Dataset1/test/000/65gr.png  
  inflating: ear recognition datasets/Dataset1/test/000/65rfb.png  
  inflating: ear recognition datasets/Dataset1/test/000/66gr.png  
  inflating: ear recognition datasets/Dataset1/test/000/66rfb.png  
  inflating: ear recognition datasets/Dataset1/test/000/67gr.png  
  inflat

In [ ]:
DATADIR = "./ear recognition datasets/Dataset1/train/"
CATEGORIES = ["000", "001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011", "012"]

In [ ]:
from __future__ import print_function

import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, classification_report

import numpy as np
import cv2
import random
import matplotlib.pyplot as plt

In [ ]:
for category in CATEGORIES:
        path = os.path.join(DATADIR, category)

        for img in os.listdir(path):
            img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            break
        break

In [ ]:
img_array.shape

(227, 227)

In [ ]:
DATAD = "./ear recognition datasets/Dataset1/test/"
CATEGORIES = ["000", "001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011", "012"]

In [ ]:
for category in CATEGORIES:
        path = os.path.join(DATAD, category)

        for img in os.listdir(path):
            img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
            break
        break

In [ ]:
img_arr.shape

(227, 227)

In [ ]:
training_data = []
testing_data = []

In [ ]:
for category in CATEGORIES:
            path = os.path.join(DATADIR, category)
            class_num = CATEGORIES.index(category)

            for img in os.listdir(path):
                try:
                    img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                    training_data.append([img_array, class_num])
                except Exception as e:
                    pass

In [ ]:
print("Training Data")
print(len(training_data))

Training Data
1560


In [ ]:
random.shuffle(training_data)

In [ ]:
a = []
b = []
for features, labels in training_data:
    a.append(features)
    b.append(labels)

a = np.array(a).reshape(-1, 227, 227, 1)
print(a.shape)
np.random.seed(1000)
X_train = a
Y_train = b
X_train = X_train/255.0

(1560, 227, 227, 1)


In [ ]:
for category in CATEGORIES:
            path = os.path.join(DATAD, category)
            class_num = CATEGORIES.index(category)

            for img in os.listdir(path):
                try:
                    img_arr = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                    testing_data.append([img_arr, class_num])
                except Exception as e:
                    pass

In [ ]:
    print("Testing Data")
    print(len(testing_data))

Testing Data
520


In [ ]:
random.shuffle(testing_data)

In [ ]:
    p = []
    q = []

    for features, labels in testing_data:
        p.append(features)
        q.append(labels)

    p = np.array(p).reshape(-1, 227, 227, 1)

    X_test = p
    Y_test = q

    X_test = X_test/255.0

In [ ]:
X_train.shape

(1560, 227, 227, 1)

In [ ]:
X_test.shape

(520, 227, 227, 1)

In [ ]:
train_Y_one_hot = to_categorical(Y_train)
test_Y_one_hot = to_categorical(Y_test)

In [ ]:
from sklearn.model_selection import train_test_split
train_X,valid_X,train_label,valid_label = train_test_split(X_train, train_Y_one_hot, test_size=0.2, random_state=13)

In [ ]:
import keras
from keras.models import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.advanced_activations import LeakyReLU

In [ ]:
batch_size = 64
epochs = 20
num_classes = 13

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),activation='linear',input_shape=(227,227,1),padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D((2, 2),padding='same'))
model.add(Conv2D(64, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Conv2D(128, (3, 3), activation='linear',padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2),padding='same'))
model.add(Flatten())
model.add(Dense(128, activation='linear'))
model.add(LeakyReLU(alpha=0.1))
model.add(Dense(num_classes, activation='softmax'))

In [ ]:
model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 227, 227, 32)      320       
_________________________________________________________________
leaky_re_lu_12 (LeakyReLU)   (None, 227, 227, 32)      0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 114, 114, 32)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 114, 114, 64)      18496     
_________________________________________________________________
leaky_re_lu_13 (LeakyReLU)   (None, 114, 114, 64)      0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 57, 57, 64)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 57, 57, 128)      

In [ ]:
fas_train = model.fit(train_X, train_label, batch_size=batch_size,epochs=epochs,verbose=1,validation_data=(valid_X, valid_label))

Epoch 1/20
7/7 [==============================] - 46s 6s/step - loss: 3.0479 - accuracy: 0.1084 - val_loss: 2.4602 - val_accuracy: 0.1538
Epoch 2/20
7/7 [==============================] - 40s 6s/step - loss: 2.3486 - accuracy: 0.2269 - val_loss: 1.9878 - val_accuracy: 0.2981
Epoch 3/20
7/7 [==============================] - 39s 6s/step - loss: 1.9343 - accuracy: 0.3263 - val_loss: 1.7429 - val_accuracy: 0.4904
Epoch 4/20
7/7 [==============================] - 39s 6s/step - loss: 1.4865 - accuracy: 0.5192 - val_loss: 1.3716 - val_accuracy: 0.5096
Epoch 5/20
7/7 [==============================] - 39s 6s/step - loss: 1.0684 - accuracy: 0.6126 - val_loss: 1.1341 - val_accuracy: 0.5962
Epoch 6/20
7/7 [==============================] - 39s 6s/step - loss: 0.7559 - accuracy: 0.7651 - val_loss: 1.0772 - val_accuracy: 0.6058
Epoch 7/20
7/7 [==============================] - 39s 6s/step - loss: 0.6274 - accuracy: 0.7885 - val_loss: 1.1782 - val_accuracy: 0.5962
Epoch 8/20
7/7 [==================

In [ ]:
scores = model.evaluate(X_test, to_categorical(Y_test))
print('Accuracy: %.3f' % scores[1])

17/17 [==============================] - 13s 767ms/step - loss: 0.0897 - accuracy: 0.9788
Accuracy: 0.979


In [ ]:
y_pred=model.predict(X_test,batch_size=32,verbose=0)
y_pred_bool=np.argmax(y_pred,axis=1)
print(classification_report(Y_test,y_pred_bool))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00        40
           1       0.93      0.95      0.94        40
           2       1.00      0.93      0.96        40
           3       0.95      1.00      0.98        40
           4       0.98      1.00      0.99        40
           5       1.00      1.00      1.00        40
           6       0.97      0.95      0.96        40
           7       1.00      1.00      1.00        40
           8       1.00      1.00      1.00        40
           9       0.95      1.00      0.98        40
          10       1.00      0.95      0.97        40
          11       1.00      1.00      1.00        40
          12       0.95      0.95      0.95        40

    accuracy                           0.98       520
   macro avg       0.98      0.98      0.98       520
weighted avg       0.98      0.98      0.98       520



In [ ]:
DATA = "./ear recognition datasets/Dataset1/validation/input"
CATEGORIES = ["000", "001", "002", "003", "004", "005", "006", "007", "008", "009", "010", "011", "012"]

In [ ]:
        p = os.path.join(DATA)
        for img in os.listdir(p):
            img_a = cv2.imread(os.path.join(p, img), cv2.IMREAD_GRAYSCALE)
            break

In [ ]:
img_a.shape

(227, 227)

In [ ]:
predict_data = []

In [ ]:
for category in CATEGORIES:
            p = os.path.join(DATA)
            class_num = CATEGORIES.index(category)

            for img in os.listdir(p):
                try:
                    img_a = cv2.imread(os.path.join(p, img), cv2.IMREAD_GRAYSCALE)
                    predict_data.append([img_a, class_num])
                except Exception as e:
                    pass

In [ ]:
print(len(predict_data))

13


In [ ]:
    a = []
    b = []

    for features, labels in predict_data:
        a.append(features)
        b.append(labels)

    a = np.array(a).astype('float')
    a = np.array(a).reshape(-1, 227, 227, 1)

    y = model.predict(a)

In [ ]:
print(y)

[[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
